# European Unique Devices by Project Family

## Task Description:

Create the following datasets and export to a csv file. 

* Unique Devices by Project Family
  * Requested fields: Project Family, 6-month average unique devices
  * Filters: Only include countries in European Union (should be available in canonical data)

## Methodology and Data Sources: 

Unique devices per project family data comes from the [unique_devices_per_project_family_monthly](https://github.com/wikimedia/analytics-refinery/blob/master/oozie/unique_devices/per_project_family/monthly/unique_devices_per_project_family_monthly.hql). 

**Metric**: The `uniques_estimate` average over the last 6 months **(February thru July 2024)**. Data limited to only countries within European Union.

**Important note**: 
- Unique devices by family metrics has been overcounted by approx ~5% globally between Feb 9, 2021 and June 30, 2022.  
For more info, read [2021-02-09 Unique Devices By Family Overcount](https://wikitech.wikimedia.org/wiki/Analytics/Data_Lake/Data_Issues/2021-02-09_Unique_Devices_By_Family_Overcount)
- Unique devices were inflated due to actors showing automated behavior (like Cloud services, VPNs etc.) which were not filtered correctly in our bot labelling. This has affected data for July, Aug and Sep 2024. See [Investigate recent rise in Unique Devices](https://phabricator.wikimedia.org/T373630) 


EU country data is based on data from [canonical_data](https://github.com/wikimedia-research/canonical-data) dataset. 

Other notes:
* Used the new `is_eu` flag from canonical_data.countries to filter EU member countries.
* The breakdown of Wikimedia projects will not be equal to Wikimedia project family which is a known consequence of [T325544](https://phabricator.wikimedia.org/T325544) . However, it is recommended to use unique_devices_per_domain_monthly numbers for the wikimedia project breakdown. 

In [4]:
import pandas as pd
import wmfdata as wmf

## Unique Devices by Project Family

In [5]:

query = '''

SELECT
    project_family,
    countries.name AS country, 
    ud.year,
    ud.month,
    SUM(ud.uniques_estimate) as unique_devices
  FROM wmf.unique_devices_per_project_family_monthly ud,
    canonical_data.countries AS countries  
  WHERE 
    ud.country_code = countries.iso_code 
  -- only European Union countries
    AND countries.is_eu  
  -- review most recent 6 months available: Feb - July 2024
    AND 
     YEAR = 2024 AND MONTH >= 02
      AND MONTH < 8
  GROUP BY 
    project_family,
    name,
    year,
    month

'''

In [6]:
unique_devices_byprojectfamily = wmf.spark.run(query)

In [7]:
unique_devices_byprojectfamily_total = unique_devices_byprojectfamily.groupby(['project_family', 'year','month']).sum()

/tmp/ipykernel_789586/957048093.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  unique_devices_byprojectfamily_total = unique_devices_byprojectfamily.groupby(['project_family', 'year','month']).sum()


In [39]:
unique_devices_byprojectfamily_avg = (unique_devices_byprojectfamily_total.groupby(['project_family']).mean().
                                          round().sort_values(by=['unique_devices'], ascending = False).reset_index())


In [40]:
unique_devices_byprojectfamily_avg['Monthly Avg Unique Users'] = (unique_devices_byprojectfamily_avg['unique_devices']/2.4).round(-3)

In [41]:
unique_devices_byprojectfamily_avg.columns = ['Project Family', 'Monthly Avg Unique Devices', 'Monthly Avg Unique Users']

In [ ]:
unique_devices_byprojectfamily_avg

In [31]:
unique_devices_byprojectfamily_avg.to_csv('unique_devices_byprojectfamily_avg.csv', index = True)

### Roll up language wikimedia domain into 'wikimedia.org'

In [21]:
query = '''

-- collect unique devices per project in the wikimedia domain and country per month

SELECT
    --wikis.domain_name AS project,
    IF(rlike (wikis.database_code,'^.{2}wikimedia'),'wikimedia.org',wikis.domain_name) AS project,
    countries.name AS country,
    ud.year,
    ud.month,
    SUM(ud.uniques_estimate) as unique_devices
FROM wmf.unique_devices_per_domain_monthly AS ud
JOIN canonical_data.countries AS countries 
    ON ud.country_code = countries.iso_code
-- combine mobile and web domains
    
-- Strip mobile subdomains so mobile and desktop sites are combined. 
LEFT JOIN canonical_data.wikis AS wikis ON
REGEXP_REPLACE(
    REGEXP_REPLACE(
        -- The canonical domains for Wikidata and MediaWiki.org start with `www`, which 
        -- gets _replaced_ by the mobile subdomain. Combine the two possibilities for each site.
        REGEXP_REPLACE(
            REGEXP_REPLACE(domain, "^m\\\\.wikidata", "www.wikidata"),
        "^m\\\\.mediawiki", "www.mediawiki"),
    "^m\\\\.", ""),
"\\\\.m\\\\.", ".")= wikis.domain_name

-- only European Union countries
    WHERE countries.is_eu = true

-- review most recent 6 months available: Feb thru July 2024
    AND YEAR = 2024 AND MONTH >= 02
      AND MONTH < 8
  AND ud.domain LIKE '%wikimedia%'
  GROUP BY 
    project,
    name,
    year,
    month
'''



In [22]:
unique_devices_wikimedia_rollup = wmf.spark.run(query)

*Note*: The breakdown of Wikimedia projects will not be equal to Wikimedia project family which is a known consequence of [T325544](https://phabricator.wikimedia.org/T325544) . But according to [T301403#8478621](https://phabricator.wikimedia.org/T301403#8478621) we understand that we can trust the `unique_devices_per_domain_monthly` numbers and shouldnt report the `unique_devices_per_project_family_monthly` numbers for **wikimedia**, which is what we're doing here. 

In [23]:
unique_devices_wikimedia_rollup_eu   = unique_devices_wikimedia_rollup.groupby(['project', 'year', 'month']).sum()

/tmp/ipykernel_789586/207635059.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  unique_devices_wikimedia_rollup_eu   = unique_devices_wikimedia_rollup.groupby(['project', 'year', 'month']).sum()


In [24]:
unique_devices_wikimedia_rollup_eu_avg = (unique_devices_wikimedia_rollup_eu.groupby(['project']).mean().
                                        round().sort_values(by=['unique_devices'], ascending = False).reset_index())



In [34]:
unique_devices_wikimedia_rollup_eu_avg['Monthly Avg Unique Users'] = (unique_devices_wikimedia_rollup_eu_avg['unique_devices']/2.4).round(-3)

In [ ]:
unique_devices_wikimedia_rollup_eu_avg

In [37]:
unique_devices_wikimedia_rollup_eu_avg.to_csv('unique_devices_wikimedia_rollup_eu_avg.csv', index = True)